<h1>Movie Reviews Analysis</h1>

<h3> Important Q&A</h3>

<h4>1. What is the business question?</h4>
<h5>What is the movie review phrases on a scale of five values: negative, somewhat negative, neutral, somewhat positive, positive?</h5>

<h4>2. What does each row represent?</h4>
<h5>Each row repersents a phrase</h5>

<h4>3. What is the evaluation method?</h4>
<h5>What is the movie review phrases on a scale of five values: negative, somewhat negative, neutral, somewhat positive, positive?</h5>


<h2>1. Imports</h2>

In [ ]:
# load data libraries
import numpy as np # linear algebra library
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import zipfile # to read zip files
from sklearn.model_selection import train_test_split


# data understanding libraries
import matplotlib.pyplot as plt # ploting library
%matplotlib inline
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from collections import Counter
import seaborn as sns



# data preparation
import re
from nltk.stem import PorterStemmer


# ADS Creation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import StandardScaler
# Modeling
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.naive_bayes import MultinomialNB

# Evaluation and Model Selection
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn import metrics
from sklearn.model_selection import learning_curve
from sklearn.model_selection import GridSearchCV

In [ ]:
pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.precision',150)
pd.options.display.float_format = '{:,.3f}'.format

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<h2>2. Load Data </h2>

In [ ]:
#unzip the files
archive_train = zipfile.ZipFile('../input/sentiment-analysis-on-movie-reviews/train.tsv.zip')

#read training tsv file 
train = pd.read_csv('../input/sentiment-analysis-on-movie-reviews/train.tsv.zip', delimiter='\t')
train.head(20)

<h4>There are a total of 4 columns</h4>

<h5>&emsp;PhraseId</h5>
<h5>&emsp;SentenceId</h5>
<h5>&emsp;Phrase</h5>

<h5>&emsp;Sentiment:</h5> 
<h6>&emsp;&emsp;&emsp;&emsp;0 - negative<br>
&emsp;&emsp;&emsp;&emsp;1 - somewhat negative<br>
&emsp;&emsp;&emsp;&emsp;2 - neutral<br>
&emsp;&emsp;&emsp;&emsp;3 - somewhat positive<br>
&emsp;&emsp;&emsp;&emsp;4 - positive</h6>

In [ ]:
#split data into train and test data
#then split the test data into test and validation 
train_data, test_data = train_test_split(train, test_size=0.4, random_state=1)
val_data, test_data = train_test_split(test_data, test_size=0.5, random_state=1)

#resets index after splitting data
train_data = train_data.reset_index(drop=True)
val_data = val_data.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)

print("Train set size is ",len(train_data))
print("Val set size is ",len(val_data))
print("Test set size is ",len(test_data))

<h3>2. Data Preparation</h3>

In [ ]:
stopwords = set(['Laura', 'Pauline', 'BMW', 'Carol', 'Kane', 'Evelyn', 'Cahill',
 'Adam',
 'Garcia',
 'X-Men',
 'Glover',
 'Disney',
 'Antwone',
 'Fisher',
 'Gayton',
 'Lyne',
 'African',
 'American',
 'Godard',
 'Dickens',
 'Woody',
 'Martin',
 'Scorsese',
 'New',
 'York',
 'Decasia',
 'Limit',
 'Hollywood',
 'Michael',
 'Moore',
 'Robin',
 'Williams',
 'Rorschach',
 'Watts',
 'Broomfield',
 'Washington',
 'Argentine',
 'Fabian',
 'Bielinsky',
 'Tunis',
 'Bebe',
 'Neuwirth',
 'John',
 'C.',
 'Walsh',
 'Ms.',
 'Fontaine',
 'Ritchie',
 'Copmovieland',
 'Egoyan',
 'France',
 'Carlen',
 'Campanella',
 'Norma',
 'Rae',
 'Bartlett',
 'Chicago',
 'Gere',
 'Peter',
 'Mattei',
 'Famuyiwa',
 'Tara',
 'Reid',
 'Hawaiian',
 'Sendak',
 'David',
 'Cronenberg',
 'Anne',
 'Busby',
 'Malkovich',
 'Statham',
 'Freudian',
 'Cattaneo',
 'Tim',
 'Apted',
 'Nicholas',
 'Kazan',
 'Kilmer',
 'One',
 'Jaglom',
 'Twohy',
 'Chinese',
 'Jerry',
 'Bruckheimer',
 'Joel',
 'Schumacher',
 'Chris',
 'Smith',
 'Hawk',
 'Katherine',
 'Blake',
 'Wilde',
 'Clint',
 'Eastwood',
 'Miramax',
 'Reno',
 'Louiso',
 'Ivory',
 'Vietnam',
 'Vietnamese',
 'Shakespeare',
 'America',
 'Del',
 'Toro',
 'Hip-Hop',
 'Scooby-Doo',
 'Aan',
 'Steve',
 'Irwin',
 'Taylor',
 'Le',
 'Jean-Luc',
 'Pamela',
 'Taymor',
 'George',
 'W.',
 'Bush',
 'Henry',
 'Kissinger',
 'Larry',
 'King',
 'Holly',
 'Marina',
 'Stanley',
 'Kwan',
 'Joseph',
 'Heller',
 'Kurt', 
 'Vonnegut',
 'Titus',
 'Minnie',
 'Allen',
 'Britney',
 'Spears',
 'Farrelly',
 'Tsai',
 'Rob',
 'Schneider',
 'Greenfingers',
 'Harmon',
 'Zhuangzhuang',
 'Eric',
 'Rohmer',
 'Blanchett',
 'Ribisi',
 'Jacquot',
 'Zaidan',
 'Wiseman',
 'Ian',
 'Holm',
 'Vincent',
 'R.',
 'Nebrida',
 'Laurice',
 'Guillen',
 'De',
 'Niro',
 'Martha',
 'Margaret',
 'Thatcher',
 'Assayas',
 'Sara',
 'Abdul',
 'Amy',
 'Neil',
 'Dana',
 'Sinise',
 'Hartley',
 'Bruce',
 'Willis',
 'Dean',
 'Zelda',
 'Tony',
 'Jackal',
 'Nelson',
 'Harlem',
 'Mick',
 'Graham',
 'Myer',
 'Koepp',
 'Wimmer',
 'Robert',
 'Travis',
 'Bickle',
 'Cockettes',
 'Jez',
 'Steven',
 'Shainberg',
 'Mary',
 'Gaitskill',
 'Reyes',
 'Roberto',
 'Benigni',
 'Kaufman',
 'Jonze',
 'Ralph',
 'Waltz',
 'Roger',
 'Kumble',
 'Damon\\/Bourne',
 'Charlie',
 'Donald',
 'Madonna'])
porter = PorterStemmer()

def ret_words(Phrase):
    Phrase = Phrase.lower()
    Phrase = Phrase.replace('-', '')
    Phrase = Phrase.replace(',', ' ')
    Phrase = Phrase.replace(';', ' ')
    Phrase = Phrase.replace('\'', '')
    Phrase = Phrase.replace('\\', '')
    Phrase = Phrase.replace('/', '')
    Phrase = Phrase.replace('.', '')
    Phrase = Phrase.replace('...', '')
    words = []
    for word in Phrase.split():
        if re.findall('[0-9]', word): continue
        if word in stopwords: continue
        if re.findall('[^a-zA-Z]',re.sub(r'[^\w\s]','',word)): continue
        if len(word) > 0: words.append(porter.stem(re.sub(r'[^\w\s]','',word)))
    return ' '.join(words)

def preprocess(df,flag):
    # add column
    df["words_num"]=df['Phrase'].str.split().str.len()
     # Remove recipes with only one Ingredient
    if flag == 0 :
        df = df.drop(df[df["words_num"]<=1].index)
    
    # Convert list of ingredients to string
    df['Phrase'] = df["Phrase"].apply(ret_words)
    
    return df

In [ ]:
train_preprocessed = preprocess(train_data,0)
test_preprocessed = preprocess(test_data,1)
val_preprocessed = preprocess(val_data,1)

In [ ]:
train_preprocessed.head(10)

<h3>3. Data Modeling</h3>

In [ ]:
id_train, X_train, y_train = train_preprocessed['PhraseId'], train_preprocessed['Phrase'], train_preprocessed['Sentiment']
id_test, X_test, y_test = test_preprocessed['PhraseId'], test_preprocessed['Phrase'], test_preprocessed['Sentiment']
id_val, X_val, y_val = val_preprocessed['PhraseId'], val_preprocessed['Phrase'], val_preprocessed['Sentiment']

<h4>3.1 BOW</h4>

In [ ]:
LR_clf_counts = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', LogisticRegression(random_state=0, max_iter=2000))
])
LR_clf_counts.fit(X_train, y_train)
LR_cnt_pred_tr = LR_clf_counts.predict(X_train)

print(accuracy_score(y_train, LR_cnt_pred_tr))
print(precision_score(y_train, LR_cnt_pred_tr, average='weighted'))

In [ ]:
# Create CV training and test scores for various training set sizes
train_sizes, train_scores, test_scores = learning_curve(LR_clf_counts, 
                                                        X_train, 
                                                        y_train,
                                                        # Number of folds in cross-validation
                                                        cv=3,
                                                        # Evaluation metric
                                                        scoring='precision_weighted',
                                                        # Use all computer cores
                                                        n_jobs=-1, 
                                                        # 50 different sizes of the training set
                                                        train_sizes=np.linspace(0.01, 1.0, 10))

# Create means and standard deviations of training set scores
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)

# Create means and standard deviations of test set scores
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)
fig, ax = plt.subplots(figsize=(15,10))
# Draw lines
plt.plot(train_sizes, train_mean, '--', color="#111111",  label="Training score")
plt.plot(train_sizes, test_mean, color="#111111", label="Cross-validation score")

# Draw bands
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, color="#DDDDDD")
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, color="#DDDDDD")

# Create plot
plt.title("Learning Curve")
plt.xlabel("Training Set Size"), plt.ylabel("Weighted Precision Score"), plt.legend(loc="best")
plt.tight_layout()
plt.show()


In [ ]:
SVM_clf_counts = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', LinearSVC(max_iter=3000))
])
SVM_clf_counts.fit(X_train, y_train)
SVM_cnt_pred_tr = SVM_clf_counts.predict(X_train)

print(accuracy_score(y_train, SVM_cnt_pred_tr))
print(precision_score(y_train, SVM_cnt_pred_tr, average='weighted'))

In [ ]:
# Create CV training and test scores for various training set sizes
train_sizes, train_scores, test_scores = learning_curve(SVM_clf_counts, 
                                                        X_train, 
                                                        y_train,
                                                        # Number of folds in cross-validation
                                                        cv=3,
                                                        # Evaluation metric
                                                        scoring='precision_weighted',
                                                        # Use all computer cores
                                                        n_jobs=-1, 
                                                        # 50 different sizes of the training set
                                                        train_sizes=np.linspace(0.01, 1.0, 10))

# Create means and standard deviations of training set scores
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)

# Create means and standard deviations of test set scores
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)
fig, ax = plt.subplots(figsize=(15,10))
# Draw lines
plt.plot(train_sizes, train_mean, '--', color="#111111",  label="Training score")
plt.plot(train_sizes, test_mean, color="#111111", label="Cross-validation score")

# Draw bands
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, color="#DDDDDD")
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, color="#DDDDDD")

# Create plot
plt.title("Learning Curve")
plt.xlabel("Training Set Size"), plt.ylabel("Weighted Precision Score"), plt.legend(loc="best")
plt.tight_layout()
plt.show()

In [ ]:
NB_clf_counts = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', MultinomialNB())
])
NB_clf_counts.fit(X_train, y_train)
NB_cnt_pred_tr = NB_clf_counts.predict(X_train)

print(accuracy_score(y_train, NB_cnt_pred_tr))
print(precision_score(y_train, NB_cnt_pred_tr, average='weighted'))

In [ ]:
# Create CV training and test scores for various training set sizes
train_sizes, train_scores, test_scores = learning_curve(NB_clf_counts, 
                                                        X_train, 
                                                        y_train,
                                                        # Number of folds in cross-validation
                                                        cv=3,
                                                        # Evaluation metric
                                                        scoring='precision_weighted',
                                                        # Use all computer cores
                                                        n_jobs=-1, 
                                                        # 50 different sizes of the training set
                                                        train_sizes=np.linspace(0.01, 1.0, 10))

# Create means and standard deviations of training set scores
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)

# Create means and standard deviations of test set scores
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)
fig, ax = plt.subplots(figsize=(15,10))
# Draw lines
plt.plot(train_sizes, train_mean, '--', color="#111111",  label="Training score")
plt.plot(train_sizes, test_mean, color="#111111", label="Cross-validation score")

# Draw bands
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, color="#DDDDDD")
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, color="#DDDDDD")

# Create plot
plt.title("Learning Curve")
plt.xlabel("Training Set Size"), plt.ylabel("Weighted Precision Score"), plt.legend(loc="best")
plt.tight_layout()
plt.show()

<h4>3.2 BOW</h4>

In [ ]:
LR_clf_tfidf = Pipeline([
    ('tfidf', TfidfVectorizer(sublinear_tf=True, min_df=5, max_df=0.25, norm='l2', encoding='latin-1',ngram_range=(1, 2), stop_words='english')),
    ('clf', LogisticRegression(random_state=0, max_iter=2000))
])
LR_clf_tfidf.fit(X_train, y_train)
LR_tfidf_pred_tr = LR_clf_tfidf.predict(X_train)

print(accuracy_score(y_train, LR_tfidf_pred_tr))
print(precision_score(y_train, LR_tfidf_pred_tr, average='weighted'))

In [ ]:
# Create CV training and test scores for various training set sizes
train_sizes, train_scores, test_scores = learning_curve(LR_clf_tfidf, 
                                                        X_train, 
                                                        y_train,
                                                        # Number of folds in cross-validation
                                                        cv=3,
                                                        # Evaluation metric
                                                        scoring='precision_weighted',
                                                        # Use all computer cores
                                                        n_jobs=-1, 
                                                        # 50 different sizes of the training set
                                                        train_sizes=np.linspace(0.01, 1.0, 10))

# Create means and standard deviations of training set scores
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)

# Create means and standard deviations of test set scores
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)
fig, ax = plt.subplots(figsize=(15,10))
# Draw lines
plt.plot(train_sizes, train_mean, '--', color="#111111",  label="Training score")
plt.plot(train_sizes, test_mean, color="#111111", label="Cross-validation score")

# Draw bands
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, color="#DDDDDD")
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, color="#DDDDDD")

# Create plot
plt.title("Learning Curve")
plt.xlabel("Training Set Size"), plt.ylabel("Weighted Precision Score"), plt.legend(loc="best")
plt.tight_layout()
plt.show()

In [ ]:
SVM_clf_tfidf = Pipeline([
    ('tfidf', TfidfVectorizer(sublinear_tf=True, min_df=5, max_df=0.25, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')),
    ('clf', LinearSVC( max_iter=2000))
])
SVM_clf_tfidf.fit(X_train, y_train)
SVM_tfidf_pred_tr = SVM_clf_tfidf.predict(X_train)

print(accuracy_score(y_train, SVM_tfidf_pred_tr))
print(precision_score(y_train, SVM_tfidf_pred_tr, average='weighted'))

In [ ]:
# Create CV training and test scores for various training set sizes
train_sizes, train_scores, test_scores = learning_curve(SVM_clf_tfidf, 
                                                        X_train, 
                                                        y_train,
                                                        # Number of folds in cross-validation
                                                        cv=3,
                                                        # Evaluation metric
                                                        scoring='precision_weighted',
                                                        # Use all computer cores
                                                        n_jobs=-1, 
                                                        # 50 different sizes of the training set
                                                        train_sizes=np.linspace(0.01, 1.0, 10))

# Create means and standard deviations of training set scores
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)

# Create means and standard deviations of test set scores
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)
fig, ax = plt.subplots(figsize=(15,10))
# Draw lines
plt.plot(train_sizes, train_mean, '--', color="#111111",  label="Training score")
plt.plot(train_sizes, test_mean, color="#111111", label="Cross-validation score")

# Draw bands
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, color="#DDDDDD")
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, color="#DDDDDD")

# Create plot
plt.title("Learning Curve")
plt.xlabel("Training Set Size"), plt.ylabel("Weighted Precision Score"), plt.legend(loc="best")
plt.tight_layout()
plt.show()

In [ ]:
NB_clf_tfidf = Pipeline([
    ('tfidf', TfidfVectorizer(sublinear_tf=True, min_df=5, max_df=0.25, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')),
    ('clf', MultinomialNB())
])
NB_clf_tfidf.fit(X_train, y_train)
NB_tfidf_pred_tr = NB_clf_tfidf.predict(X_train)

print(accuracy_score(y_train, NB_tfidf_pred_tr))
print(precision_score(y_train, NB_tfidf_pred_tr, average='weighted'))

In [ ]:
# Create CV training and test scores for various training set sizes
train_sizes, train_scores, test_scores = learning_curve(NB_clf_tfidf, 
                                                        X_train, 
                                                        y_train,
                                                        # Number of folds in cross-validation
                                                        cv=3,
                                                        # Evaluation metric
                                                        scoring='precision_weighted',
                                                        # Use all computer cores
                                                        n_jobs=-1, 
                                                        # 50 different sizes of the training set
                                                        train_sizes=np.linspace(0.01, 1.0, 10))

# Create means and standard deviations of training set scores
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)

# Create means and standard deviations of test set scores
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)
fig, ax = plt.subplots(figsize=(15,10))
# Draw lines
plt.plot(train_sizes, train_mean, '--', color="#111111",  label="Training score")
plt.plot(train_sizes, test_mean, color="#111111", label="Cross-validation score")

# Draw bands
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, color="#DDDDDD")
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, color="#DDDDDD")

# Create plot
plt.title("Learning Curve")
plt.xlabel("Training Set Size"), plt.ylabel("Weighted Precision Score"), plt.legend(loc="best")
plt.tight_layout()
plt.show()

<h4>Hyperparameter tuning</h4>

In [ ]:
vect=  CountVectorizer()
X_train_cnt = vect.fit_transform(X_train)

<h4>Logistic Regression</h4>

In [ ]:
def LR_param_selection(X, y, nfolds):
    Cs = [0.01, 0.1, 1, 10]
    param_grid = {'C': Cs}
    grid_search = GridSearchCV(LogisticRegression(random_state=0,max_iter=2000), param_grid, cv=nfolds)
    grid_search.fit(X, y)
    grid_search.best_params_
    return grid_search.best_params_

In [ ]:
LR_param_selection( X_train_cnt,y_train,2)

In [ ]:
LR_clf_counts = Pipeline([('vect', CountVectorizer()),
                   ('clf', LogisticRegression(C=1,random_state=0, max_iter=2000)),
                  ])
LR_clf_counts.fit(X_train, y_train)
LR_cnt_pred_tr = LR_clf_counts.predict(X_train)
LR_cnt_pred_val = LR_clf_counts.predict(X_val)

print("accuracy on training: ",accuracy_score(y_train, LR_cnt_pred_tr))
print("precision on training: ",precision_score(y_train, LR_cnt_pred_tr, average='micro'))
print("precision on validation: ",precision_score(y_val, LR_cnt_pred_val, average='micro'))

<h3>Final Model</h3>

In [ ]:
LR_clf_counts = Pipeline([('vect', CountVectorizer()),
                   ('clf', LogisticRegression(C=1,random_state=0, max_iter=2000)),
                  ])
LR_clf_counts.fit(X_train, y_train)
LR_cnt_pred_tr = LR_clf_counts.predict(X_train)
LR_cnt_pred_val = LR_clf_counts.predict(X_val)
LR_cnt_pred_tst = LR_clf_counts.predict(X_test)


print("precision on training: ",precision_score(y_train, LR_cnt_pred_tr, average='micro'))
print("precision on validation: ",precision_score(y_val, LR_cnt_pred_val, average='micro'))
print("precision on testing: ",precision_score(y_test, LR_cnt_pred_tst, average='micro'))

In [ ]:
# Create CV training and test scores for various training set sizes
train_sizes, train_scores, test_scores = learning_curve(LR_clf_counts, 
                                                        X_train, 
                                                        y_train,
                                                        # Number of folds in cross-validation
                                                        cv=3,
                                                        # Evaluation metric
                                                        scoring='precision_weighted',
                                                        # Use all computer cores
                                                        n_jobs=-1, 
                                                        # 50 different sizes of the training set
                                                        train_sizes=np.linspace(0.01, 1.0, 10))

# Create means and standard deviations of training set scores
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)

# Create means and standard deviations of test set scores
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)
fig, ax = plt.subplots(figsize=(15,10))

# Draw lines
plt.plot(train_sizes, train_mean, '--', color="#111111",  label="Training score")
plt.plot(train_sizes, test_mean, color="#111111", label="Cross-validation score")

# Draw bands
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, color="#DDDDDD")
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, color="#DDDDDD")

# Create plot
plt.title("Learning Curve")
plt.xlabel("Training Set Size"), plt.ylabel("Weighted Precision Score"), plt.legend(loc="best")
plt.tight_layout()
plt.show()

<h3>Logistic Regression</h3>

In [ ]:
labels = train['Sentiment'].unique()

In [ ]:
conf_mat = pd.DataFrame(confusion_matrix(y_val, LR_cnt_pred_val, labels=labels))
conf_mat_n = conf_mat.divide(conf_mat.sum(axis=1), axis=0)
fig, ax = plt.subplots(figsize=(20,12))
sns.heatmap(conf_mat_n, annot=True,xticklabels=labels,yticklabels=labels)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

In [ ]:
Predicted_vals = pd.DataFrame({'id' : id_val, 'Sentiment': X_val, 'Actual': y_val, 'Preds': LR_cnt_pred_val})

<h2>Model Run:</h2>

In [ ]:
#read
archive_train = zipfile.ZipFile('../input/sentiment-analysis-on-movie-reviews/train.tsv.zip')
archive_test = zipfile.ZipFile('../input/sentiment-analysis-on-movie-reviews/test.tsv.zip')


#read training tsv file 
final_train = pd.read_csv('../input/sentiment-analysis-on-movie-reviews/train.tsv.zip', delimiter='\t')
final_test = pd.read_csv('../input/sentiment-analysis-on-movie-reviews/test.tsv.zip', delimiter='\t')

In [ ]:
#prepare
ftrain_preprocessed = preprocess(final_train,0)
ftest_preprocessed = preprocess(final_test,1)

In [ ]:
id_train, X_train, y_train = ftrain_preprocessed['PhraseId'], ftrain_preprocessed['Phrase'], ftrain_preprocessed['Sentiment']
id_test, X_test, = ftest_preprocessed['PhraseId'], ftest_preprocessed['Phrase']

In [ ]:
#LR
LR_clf = Pipeline([('vect', CountVectorizer()),
                   ('clf', LogisticRegression(C=1,random_state=0, max_iter=2000)),
                  ])
LR_clf.fit(X_train, y_train)
pred_test = LR_clf_counts.predict(X_test)

In [ ]:
output=pd.DataFrame({'PhraseId':id_test, 'Sentiment':pred_test})
output.to_csv('Sentiment_preds_LR.csv',index= False)